In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
from transformers import AutoModel
from transformers import AutoTokenizer
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
from torch.optim import AdamW
from tqdm.auto import tqdm

# **Load Data**

In [ ]:
train = pd.read_csv("./Data/Preprocessed/ViCTSD_train-clean.csv")
val = pd.read_csv("./Data/Preprocessed/ViCTSD_valid-clean.csv")
test = pd.read_csv("./Data/Preprocessed/ViCTSD_test-clean.csv")

X_train = train['Comment_clean'].tolist()
y_train = train['Constructiveness'].tolist()

X_val = val['Comment_clean'].tolist()
y_val = val['Constructiveness'].tolist()

X_test = test['Comment_clean'].tolist()
y_test = test['Constructiveness'].tolist()

# **Tokenize**

In [7]:
model_name = "vinai/phobert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize dữ liệu
train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(X_val, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=128)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [8]:
# --- 3. TẠO DATASET ---
class ViCTSDDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Tạo object Dataset
train_dataset = ViCTSDDataset(train_encodings, y_train)
val_dataset = ViCTSDDataset(val_encodings, y_val)
test_dataset = ViCTSDDataset(test_encodings, y_test)

# **Model**

## **Last hidden state**

In [ ]:
class PhoBertBinary(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        self.phobert = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask):
        # 1. Qua PhoBERT
        outputs = self.phobert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        # 2. Lấy vector đặc trưng (CLS token)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)

        # 3. Qua lớp phân loại
        logits = self.classifier(pooled_output)

        return logits

## **4 last hidden state**

In [ ]:
class PhoBertBinary_Last4Layers(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        self.phobert = AutoModel.from_pretrained(model_name, output_hidden_states=True)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask):
        # 2. Qua PhoBERT
        outputs = self.phobert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        # 3. Lấy Hidden States
        all_hidden_states = outputs.hidden_states

        # 4. Trích xuất 4 lớp cuối cùng
        stack_last_4 = torch.stack(all_hidden_states[-4:])
        sum_last_4 = torch.sum(stack_last_4, dim=0)
        # cls_output size: [batch_size, 768]
        cls_output = sum_last_4[:, 0, :]

        cls_output = self.dropout(cls_output)
        logits = self.classifier(cls_output)

        return logits

train loop

In [21]:
def train_epoch(model, dataloader, optimizer, criterion, device):
    model.train() # Chuyển sang chế độ train (bật dropout)
    total_loss = 0
    correct = 0
    total = 0

    # tqdm giúp hiện thanh tiến trình
    progress_bar = tqdm(dataloader, desc="Training", leave=False)

    for batch in progress_bar:
        # 1. Chuyển dữ liệu sang device (GPU/CPU)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # 2. Reset gradients
        optimizer.zero_grad()

        # 3. Forward pass (Lan truyền xuôi)
        # Lưu ý: Class PhoBertBinary của bạn nhận input_ids và attention_mask
        outputs = model(input_ids, attention_mask)

        # 4. Tính Loss
        loss = criterion(outputs, labels)
        total_loss += loss.item()

        # 5. Backward pass (Lan truyền ngược)
        loss.backward()

        # 6. Cập nhật trọng số
        optimizer.step()

        # Tính độ chính xác sơ bộ
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Cập nhật thanh tiến trình
        progress_bar.set_postfix({'loss': loss.item()})

    avg_loss = total_loss / len(dataloader)
    return avg_loss

def evaluate(model, dataloader, criterion, device):
    model.eval() # Chuyển sang chế độ eval (tắt dropout)
    total_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad(): # Không tính gradient để tiết kiệm bộ nhớ
        for batch in tqdm(dataloader, desc="Evaluating", leave=False):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            _, predicted = torch.max(outputs, 1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_loss = total_loss / len(dataloader)
    # Dùng sklearn để tính các chỉ số chi tiết
    acc = accuracy_score(all_labels, all_preds)
    report = classification_report(all_labels, all_preds, digits=4)

    return avg_loss, acc, report

# **Main**

In [ ]:
# 1. Thiết lập tham số
BATCH_SIZE = 16 # Hoặc 32 tùy vào VRAM của bạn
EPOCHS = 3      # Thường fine-tune chỉ cần 3-5 epochs
LEARNING_RATE = 2e-5 # Learning rate chuẩn cho BERT/PhoBERT

# 2. Tạo DataLoader
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [22]:
# Setup thiết bị
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Khởi tạo mô hình
model = PhoBertBinary(model_name="vinai/phobert-base")
model.to(device) # Chuyển mô hình sang GPU

# Khai báo Optimizer (AdamW là chuẩn cho Transformer)
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

# Hàm Loss (CrossEntropyLoss đã bao gồm Softmax bên trong)
criterion = nn.CrossEntropyLoss()

print("Start Training...")
for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch + 1}/{EPOCHS}")

    train_loss = train_epoch(model, train_loader, optimizer, criterion, device)
    print(f"Average Train Loss: {train_loss:.4f}")

Using device: cuda
Start Training...

Epoch 1/3


Average Train Loss: 0.4288

Epoch 2/3


Average Train Loss: 0.3611

Epoch 3/3


Average Train Loss: 0.2950


In [25]:
# Setup thiết bị
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Khởi tạo mô hình
model = PhoBertBinary_Last4Layers(model_name="vinai/phobert-base")
model.to(device) # Chuyển mô hình sang GPU

# Khai báo Optimizer (AdamW là chuẩn cho Transformer)
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

# Hàm Loss (CrossEntropyLoss đã bao gồm Softmax bên trong)
criterion = nn.CrossEntropyLoss()

print("Start Training...")
for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch + 1}/{EPOCHS}")

    train_loss = train_epoch(model, train_loader, optimizer, criterion, device)
    print(f"Average Train Loss: {train_loss:.4f}")

Using device: cuda
Start Training...

Epoch 1/3


Average Train Loss: 0.4296

Epoch 2/3


Average Train Loss: 0.3427

Epoch 3/3


Average Train Loss: 0.2667


# **Evaluate**

In [23]:
# --- 1. Đánh giá trên tập Validation (Val) ---
print("\n-------------------------------")
print("Đánh giá trên tập VAL:")
val_loss, val_acc, val_report = evaluate(model, val_loader, criterion, device)

print(f"Val Loss: {val_loss:.4f}")
print(f"Val Accuracy: {val_acc:.4f}")
print("\nChi tiết báo cáo (Precision, Recall, F1):")
print(val_report)

# --- 2. Đánh giá trên tập Test ---
print("\n-------------------------------")
print("Đánh giá trên tập TEST:")
test_loss, test_acc, test_report = evaluate(model, test_loader, criterion, device)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")
print("\nChi tiết báo cáo (Precision, Recall, F1):")
print(test_report)


-------------------------------
Đánh giá trên tập VAL:


Val Loss: 0.3832
Val Accuracy: 0.8190

Chi tiết báo cáo (Precision, Recall, F1):
              precision    recall  f1-score   support

           0     0.8410    0.8820    0.8610      1271
           1     0.7751    0.7092    0.7407       729

    accuracy                         0.8190      2000
   macro avg     0.8080    0.7956    0.8008      2000
weighted avg     0.8170    0.8190    0.8171      2000


-------------------------------
Đánh giá trên tập TEST:


Test Loss: 0.3685
Test Accuracy: 0.8410

Chi tiết báo cáo (Precision, Recall, F1):
              precision    recall  f1-score   support

           0     0.8554    0.9025    0.8783       636
           1     0.8116    0.7335    0.7706       364

    accuracy                         0.8410      1000
   macro avg     0.8335    0.8180    0.8245      1000
weighted avg     0.8395    0.8410    0.8391      1000



4 last hidden state

In [26]:
# --- 1. Đánh giá trên tập Validation (Val) ---
print("\n-------------------------------")
print("Đánh giá trên tập VAL:")
val_loss, val_acc, val_report = evaluate(model, val_loader, criterion, device)

print(f"Val Loss: {val_loss:.4f}")
print(f"Val Accuracy: {val_acc:.4f}")
print("\nChi tiết báo cáo (Precision, Recall, F1):")
print(val_report)

# --- 2. Đánh giá trên tập Test ---
print("\n-------------------------------")
print("Đánh giá trên tập TEST:")
test_loss, test_acc, test_report = evaluate(model, test_loader, criterion, device)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")
print("\nChi tiết báo cáo (Precision, Recall, F1):")
print(test_report)


-------------------------------
Đánh giá trên tập VAL:


Val Loss: 0.4004
Val Accuracy: 0.8160

Chi tiết báo cáo (Precision, Recall, F1):
              precision    recall  f1-score   support

           0     0.8592    0.8497    0.8544      1271
           1     0.7429    0.7572    0.7500       729

    accuracy                         0.8160      2000
   macro avg     0.8011    0.8035    0.8022      2000
weighted avg     0.8168    0.8160    0.8164      2000


-------------------------------
Đánh giá trên tập TEST:


Test Loss: 0.3781
Test Accuracy: 0.8290

Chi tiết báo cáo (Precision, Recall, F1):
              precision    recall  f1-score   support

           0     0.8673    0.8632    0.8652       636
           1     0.7629    0.7692    0.7661       364

    accuracy                         0.8290      1000
   macro avg     0.8151    0.8162    0.8157      1000
weighted avg     0.8293    0.8290    0.8291      1000

